In [16]:
import torch
import numpy as np

from torch import nn
from torch import optim

In [17]:
epochs = 100
learning_rate = 0.1
num_layers = 1
sequence_length = 10 ## sentence를 자르기 위한 길이로 각각의 sequence data piece가 같는 길이.

In [18]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

print(len(sentence))

180


In [19]:
char_set = sorted(list(set(sentence)))
char_dict = {c : i for i, c in enumerate(char_set)}

print(len(char_dict))
print(char_set)
print(char_dict)

25
[' ', "'", ',', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'w', 'y']
{' ': 0, "'": 1, ',': 2, '.': 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'w': 23, 'y': 24}


In [20]:
input_dim = len(char_dict)
hidden_dim = len(char_dict)

print(input_dim)

25


In [21]:
## data setting
x_data, y_data = [], []
for i in range(0, len(sentence) - sequence_length): ## 180-10=170
    x_str = sentence[i : i + sequence_length] ## ex) hell
    y_str = sentence[i + 1 : i + sequence_length + 1] ## ex) ello

    x_data.append([char_dict[c] for c in x_str])
    y_data.append([char_dict[c] for c in y_str])

print(len(x_data), len(y_data))
print(x_data)
print(y_data)


170 170
[[12, 9, 0, 24, 17, 22, 0, 23, 4, 16], [9, 0, 24, 17, 22, 0, 23, 4, 16, 21], [0, 24, 17, 22, 0, 23, 4, 16, 21, 0], [24, 17, 22, 0, 23, 4, 16, 21, 0, 21], [17, 22, 0, 23, 4, 16, 21, 0, 21, 17], [22, 0, 23, 4, 16, 21, 0, 21, 17, 0], [0, 23, 4, 16, 21, 0, 21, 17, 0, 5], [23, 4, 16, 21, 0, 21, 17, 0, 5, 22], [4, 16, 21, 0, 21, 17, 0, 5, 22, 12], [16, 21, 0, 21, 17, 0, 5, 22, 12, 14], [21, 0, 21, 17, 0, 5, 22, 12, 14, 7], [0, 21, 17, 0, 5, 22, 12, 14, 7, 0], [21, 17, 0, 5, 22, 12, 14, 7, 0, 4], [17, 0, 5, 22, 12, 14, 7, 0, 4, 0], [0, 5, 22, 12, 14, 7, 0, 4, 0, 20], [5, 22, 12, 14, 7, 0, 4, 0, 20, 11], [22, 12, 14, 7, 0, 4, 0, 20, 11, 12], [12, 14, 7, 0, 4, 0, 20, 11, 12, 18], [14, 7, 0, 4, 0, 20, 11, 12, 18, 2], [7, 0, 4, 0, 20, 11, 12, 18, 2, 0], [0, 4, 0, 20, 11, 12, 18, 2, 0, 7], [4, 0, 20, 11, 12, 18, 2, 0, 7, 17], [0, 20, 11, 12, 18, 2, 0, 7, 17, 16], [20, 11, 12, 18, 2, 0, 7, 17, 16, 1], [11, 12, 18, 2, 0, 7, 17, 16, 1, 21], [12, 18, 2, 0, 7, 17, 16, 1, 21, 0], [18, 2, 0, 7, 1

In [22]:
## One-hot Encoding
x_one_hot = np.array([np.eye(input_dim)[x] for x in x_data])
print(x_one_hot.shape) ## [Batch_size, Sequence_length, input_dim]
print(x_one_hot)

(170, 10, 25)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 1.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.

In [23]:
X = torch.tensor(x_one_hot, dtype=torch.float32) ## [Batch_size, Sequence_length, input_dim]
Y = torch.tensor(y_data, dtype=torch.int64) ## [Batch_size, Sequence_length]

print(X.shape, Y.shape)

torch.Size([170, 10, 25]) torch.Size([170, 10])


In [24]:
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(RNN, self).__init__()
        """
        https://pytorch.org/docs/stable/generated/torch.nn.RNN.html RNN doc
        input_size : input x's features
        hidden_size : hidden state's features
        batch_first : If True, then the input and output tensors are provided as (batch, seq_len, feature) instead of (seq_len, batch, feature).
                      this does not apply to hidden or cell states.
        nonlinearity : The non-linearity to use. Can be either 'tanh' or 'relu'. Default: 'tanh'
        bidirectional : If True, becomes a bidirectional RNN. Default: False
                        if bidirectional=True -> D=2 else D=1

            - input : tensor of shape (seq_len, input_dim) for unbatched input, (seq_len, batch_size, input_dim) when batch_first=False
                      when batch_first=True : (batch_size, seq_len, input_dim)
            - h_0 : tensor of shape (bidirectional D * num_layers, hidden_dim) for unbatched input (bidirectional D * num_layers, batch_size, hidden_dim)
            - batch_first=True로 설정되는 경우 각 배치 원소들을 독립적으로 처리해야 브로드캐스팅을 통한 x_tW_x + h_t_1W_h 덧셈 연산이 가능하다.
            
        """
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        output, hidden_state = self.rnn(x) ## output.shape : [170, 10, 25], hidden_state.shape : [1, 170, 25]
        print(output.shape, hidden_state.shape)
        output = self.fc(output)
        return output
    
    def print_parameters(self):
        for name, param in self.named_parameters():
            print(f"{name}: {param.shape}")

In [25]:
model = RNN(input_dim, hidden_dim, num_layers)
model.print_parameters()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), learning_rate)

rnn.weight_ih_l0: torch.Size([25, 25])
rnn.weight_hh_l0: torch.Size([25, 25])
rnn.bias_ih_l0: torch.Size([25])
rnn.bias_hh_l0: torch.Size([25])
fc.weight: torch.Size([25, 25])
fc.bias: torch.Size([25])


In [26]:
for i in range(epochs):
    optimizer.zero_grad()
    outputs = model(X)
    
    loss = criterion(outputs.view(-1, input_dim), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    results_str = ""
    for j, result in enumerate(results):
        if j == 0:
            results_str += ''.join([char_set[x] for x in result])
        else:
            results_str += char_set[result[-1]]

    print(f"epoch{i} | loss : {loss.item()}")
    print(results_str + '\n')

### "if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea."

torch.Size([170, 10, 25]) torch.Size([1, 170, 25])
epoch0 | loss : 3.2136778831481934
' ''.'.'.'..'' ''.''''.''.''.' '..'' '''''.'''''''''''.'' ''' '.''''.''.''p'''p'''..'''' ''.''''''''''''''p'..'''''.' ' '' ''''''.'''''''.''''''' 'k''.''''''''.'''''p.'.' ''''.'''

torch.Size([170, 10, 25]) torch.Size([1, 170, 25])
epoch1 | loss : 2.834951639175415
                                                                                                                                                                                   

torch.Size([170, 10, 25]) torch.Size([1, 170, 25])
epoch2 | loss : 2.8961830139160156
edaoh,oolcodhrdu,fr fo hrp,odm drolo fod,,d,ol,odfo lmohmhfo fo kr lororfo k,odopcodlo k.od,ohmodh l, lmdl,ohk, lc dfolcrrh,odfoeloedm,orlo kndhrdfohmrhgmrel,ohmmrd,.o fo fo lrohr

torch.Size([170, 10, 25]) torch.Size([1, 170, 25])
epoch3 | loss : 2.6866440773010254
eoeo o o do o o toeoaoaoeo  oo o ooe o ao o o  o oo o oeeoao oo oeo hoo ooao o o o ho o  o ooeo oo o oo o  o ho o 